In [1]:
import tensorflow as tf
from cython_weave import *
import tensorflow_weave_help
reload(tensorflow_weave_help)
from tensorflow_weave_help import *
from tensorflow.python.client import timeline
from fancy_conv import *

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
weave_param = {'num_zeros': 2, 'filter_size':3}
num_image = 10
num_filters = 40
N = 32
x = np.arange(1,num_image*num_filters*N*N +1).reshape(num_image, num_filters,N,N).astype(float)

zero_Indexor = tf.convert_to_tensor(create_part_I_zero_weave_matrix((num_filters,N,N), weave_param))
array_Indexor = tf.convert_to_tensor(create_part_I_array_weave_matrix((num_filters,N,N), weave_param,include_center = 0))

a = tf.placeholder(tf.float32,(None, num_filters,N,N))
a_cat = tf.concat([a, tf.zeros_like(a)], axis = 3)

def fn_zero(x):
    return tf.gather_nd(x, zero_Indexor)

def fn_array(x):
    return tf.gather_nd(x, array_Indexor)

output = (tf.map_fn(fn_zero, a_cat), tf.map_fn(fn_array, a_cat))

with tf.Session() as sess:
    r_zero, r_array = sess.run(output, feed_dict={a:x})

In [8]:
print(r_zero.shape)
print(r_array.shape)

(10, 7, 94, 94)
(10, 7, 94, 94)


In [9]:
cython_r_zero, _ = zero_weave_forward(x, weave_param)
cython_r_array, _ = array_weave_fast_forward(x,weave_param,include_center=0)
print(cython_r_array.shape)
print(cython_r_zero.shape)

(10, 7, 94, 94)
(10, 7, 94, 94)


In [10]:
print((cython_r_zero == r_zero).mean())
print((cython_r_array==r_array).mean())

1.0
1.0


In [11]:
print(r_array[0,0,0])
print(cython_r_array[0,0,0])

[ 0.  4.  0.  0.  5.  0.  0.  6.  1.  0.  7.  2.  0.  8.  3.  0.  9.  4.
  0. 10.  5.  0. 11.  6.  0. 12.  7.  0. 13.  8.  0. 14.  9.  0. 15. 10.
  0. 16. 11.  0. 17. 12.  0. 18. 13.  0. 19. 14.  0. 20. 15.  0. 21. 16.
  0. 22. 17.  0. 23. 18.  0. 24. 19.  0. 25. 20.  0. 26. 21.  0. 27. 22.
  0. 28. 23.  0. 29. 24.  0. 30. 25.  0. 31. 26.  0. 32. 27.  0.  0. 28.
  0.  0. 29.  0.]
[ 0.  4.  0.  0.  5.  0.  0.  6.  1.  0.  7.  2.  0.  8.  3.  0.  9.  4.
  0. 10.  5.  0. 11.  6.  0. 12.  7.  0. 13.  8.  0. 14.  9.  0. 15. 10.
  0. 16. 11.  0. 17. 12.  0. 18. 13.  0. 19. 14.  0. 20. 15.  0. 21. 16.
  0. 22. 17.  0. 23. 18.  0. 24. 19.  0. 25. 20.  0. 26. 21.  0. 27. 22.
  0. 28. 23.  0. 29. 24.  0. 30. 25.  0. 31. 26.  0. 32. 27.  0.  0. 28.
  0.  0. 29.  0.]
